# Tutorial 01b: Azure AI Foundry Agent - Enterprise-Ready Travel Assistant

##  Learning Objectives
By the end of this notebook, you will:
- Understand what Azure AI Foundry (formerly Azure AI Studio) is
- Learn the differences between Azure OpenAI direct and Azure AI Foundry agents
- Create agents using Azure AI Foundry with enterprise features
- Work with Azure authentication and credentials
- Configure Azure AI projects and deployments

##  Prerequisites

Before starting this tutorial, ensure you have:

1. **Azure subscription** 
2. **Azure AI Foundry project** 
3. **Azure CLI installed** - For authentication
4. **Model deployment** in your Azure AI project (gpt-4, gpt-4o, or gpt-35-turbo)
5. **Virtual environment** set up from Tutorial 01

---

##  Setup Instructions

### Install Azure AI Package

If you followed Tutorial 01, you already have the base packages. Now add Azure AI Foundry support:

```bash
# Option 1: Using pip
pip install agent-framework-azure-ai azure-identity

# Option 2: Using uv
uv pip install agent-framework-azure-ai azure-identity
```

### Azure CLI Setup

```bash
# Install Azure CLI (if not already installed)
# macOS: 
brew install azure-cli

# Windows: Download from
# https://aka.ms/installazurecliwindows

# Linux:
curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash

# Login to Azure
az login
```

### Get Your Azure AI Foundry Credentials

1. Go to [Azure AI Foundry](https://ai.azure.com/)
2. Open your project
3. Click **"Project Settings"** → Copy the **Project Endpoint**
4. Go to **"Deployments"** → Note your **Model Deployment Name**

---

##  Key Concepts

### What is Azure AI Foundry?
Azure AI Foundry is Microsoft's enterprise AI platform that provides:
- **Managed AI Models** - Host and scale models with enterprise SLAs
- **Enterprise Security** - VNET, RBAC, managed identities, private endpoints
- **Built-in Tools** - Code interpreter, file search, Bing grounding
- **Content Safety** - Automatic content filtering and safety checks
- **Monitoring** - Built-in logging, metrics, and distributed tracing
- **Compliance** - Meet enterprise compliance requirements (SOC 2, HIPAA, etc.)

### Azure OpenAI Direct vs Azure AI Foundry

| Feature | Azure OpenAI Direct | Azure AI Foundry |
|---------|---------------------|------------------|
| **Setup** | Endpoint + API key | Azure subscription + AI project |
| **Security** | API key authentication | Enterprise security (RBAC, VNET, Managed Identity) |
| **Tools** | Function calling only | Functions + Code Interpreter + File Search + Bing |
| **Monitoring** | Basic metrics | Full Azure Monitor integration with tracing |
| **Cost** | Pay-per-token | Pay-per-token + project overhead |
| **Compliance** | Azure compliance | Full Azure compliance + AI governance |
| **Content Safety** | Manual configuration | Automatic built-in filtering |
| **Best For** | Simple applications | Enterprise production workloads |

**Tutorial 01** used **Azure OpenAI Direct** (simple, API key-based)  
**This tutorial** uses **Azure AI Foundry** (enterprise-ready, full platform)

---

## Step 1: Verify Installation

Let's verify that all required packages for Azure AI Foundry are installed.

## Step 2: Import Required Libraries

Import the Agent Framework and Azure-specific components.

In [1]:
# Verify required packages are installed
import sys

required_packages = [
    'agent_framework',
    'azure.identity',
    'dotenv'
]

print("🔍 Checking Azure AI Foundry packages...\n")

missing_packages = []
for package in required_packages:
    try:
        __import__(package.replace('-', '_'))
        print(f" {package} - installed")
    except ImportError:
        print(f" {package} - missing")
        missing_packages.append(package)

if missing_packages:
    print(f"\n  Missing packages: {', '.join(missing_packages)}")
    print("Please install them using:")
    print(f"  pip install agent-framework-azure-ai azure-identity python-dotenv")
else:
    print("\n All required packages are installed!")
    print(f" Python version: {sys.version.split()[0]}")

🔍 Checking Azure AI Foundry packages...

 agent_framework - installed
 azure.identity - installed
 dotenv - installed

 All required packages are installed!
 Python version: 3.12.12


In [2]:
import os
from random import randint, choice
from typing import Annotated

from agent_framework import ChatAgent
from agent_framework_azure_ai import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential, DefaultAzureCredential
from pydantic import Field
from dotenv import load_dotenv

load_dotenv()
print("Imports successful!")

Imports successful!


## Step 3: Configure Azure AI Foundry Connection

You need your Azure AI Foundry project information.

### 📋 Required Configuration

Add these to your `.env` file:

```bash
# Azure AI Foundry Configuration
AZURE_AI_PROJECT_ENDPOINT=https://your-project.services.ai.azure.com/api/projects/your-project-name
AZURE_AI_MODEL_DEPLOYMENT_NAME=gpt-4  # or gpt-4o, gpt-35-turbo
```

### How to Get These Values:

1. Go to [Azure AI Foundry Portal](https://ai.azure.com/)
2. Open your project
3. **Project Endpoint**: 
   - Click **"Project Settings"** in the left menu
   - Copy the **"Project connection string"** or **"Endpoint URL"**
4. **Model Deployment Name**: 
   - Go to **"Deployments"** in the left menu
   - Find your model deployment (e.g., gpt-4, gpt-4o)
   - Copy the **deployment name** (not the model name!)

### Alternative: Connection String

You can also use a connection string format:

```bash
AZURE_AI_PROJECT_CONNECTION_STRING=your-connection-string-from-ai-foundry
```

In [3]:
# Load Azure AI Foundry configuration from environment
project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
model_deployment = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME")

# Verify configuration
if project_endpoint and model_deployment:
    print(" Azure AI Foundry configuration loaded!")
    print(f"   Project Endpoint: {project_endpoint}")
    print(f"   Model Deployment: {model_deployment}")
else:
    print(" ERROR: Missing Azure AI Foundry configuration!")
    print("\nPlease add to your .env file:")
    print("  AZURE_AI_PROJECT_ENDPOINT=https://your-project.services.ai.azure.com/api/projects/your-project")
    print("  AZURE_AI_MODEL_DEPLOYMENT_NAME=gpt-4")
    print("\nOr set them directly in this notebook:")
    print("  project_endpoint = 'your-endpoint'")
    print("  model_deployment = 'your-deployment-name'")

 Azure AI Foundry configuration loaded!
   Project Endpoint: https://gk-agent-framework-project.services.ai.azure.com/api/projects/agentframworkProject
   Model Deployment: gpt-4.1


## Step 4: Understanding Azure Authentication

**Key Difference from Tutorial 01:** Azure AI Foundry uses **Azure Active Directory (Entra ID)** for authentication instead of API keys!

### Why Azure AD Instead of API Keys?

-  **More Secure** - No secrets in code, tokens auto-rotate
-  **RBAC Integration** - Fine-grained access control
-  **Audit Trail** - Who accessed what and when
-  **Managed Identity** - No credentials needed for Azure-hosted apps

### Authentication Options:

1. **AzureCliCredential** ( Best for local development)
   - Uses your `az login` session
   - No configuration needed
   - Recommended for this tutorial

2. **DefaultAzureCredential** ( Best for production)
   - Tries multiple authentication methods in order
   - Works in Azure, local development, and CI/CD
   - Recommended for deployed applications

3. **ManagedIdentityCredential** ( Best for Azure-hosted apps)
   - Uses Azure Managed Identity
   - No secrets to manage at all!
   - Perfect for Azure Container Apps, Functions, etc.

### First, Check Your Azure Authentication

In [4]:
# Verify Azure CLI authentication
async def check_auth():
    try:
        async with AzureCliCredential() as credential:
            # Try to get a token for Azure Management
            token = await credential.get_token("https://management.azure.com/.default")
            if token:
                print(" Successfully authenticated with Azure!")
                print("   Authentication method: Azure CLI (az login)")
                print("   Ready to use Azure AI Foundry")
                return True
    except Exception as e:
        print(f" Authentication failed: {e}")
        print("\n To fix this, run in your terminal:")
        print("   az login")
        print("\nThen re-run this cell.")
        return False

await check_auth()

 Successfully authenticated with Azure!
   Authentication method: Azure CLI (az login)
   Ready to use Azure AI Foundry


True

In [5]:
# Create your first Azure AI Foundry agent
async def create_basic_agent():
    print(" Creating Azure AI Foundry agent...\n")
    
    # Use async with for automatic lifecycle management
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            async_credential=credential,
            project_endpoint=project_endpoint,
            model_deployment_name=model_deployment
        ).create_agent(
            name="AzureTravelAssistant",
            instructions="""
            You are a friendly and knowledgeable travel assistant powered by Azure AI Foundry.
            Help users plan trips, answer questions about destinations, and provide travel advice.
            
            Be enthusiastic, specific, and consider factors like budget, time of year, and interests.
            """
        ) as agent,
    ):
        # Agent is created in Azure AI Foundry and ready to use!
        print(" Agent created successfully in Azure AI Foundry")
        print(f"   Agent name: {agent.name}\n")
        
        # Ask a question
        query = "What are the best destinations for a winter vacation in Europe?"
        print(f" User: {query}\n")
        
        response = await agent.run(query)
        print(f" Agent: {response}")
        
        return response

# Run the agent (only if configured)
if project_endpoint and model_deployment:
    await create_basic_agent()
else:
    print(" Please configure your Azure AI Foundry project first (see Step 3)")

 Creating Azure AI Foundry agent...

 Agent created successfully in Azure AI Foundry
   Agent name: AzureTravelAssistant

 User: What are the best destinations for a winter vacation in Europe?

 Agent: Europe is magical in winter, and there are destinations to suit every style—whether you crave snowy mountain adventures, festive cities, or mild escapes. Here are some of the **best destinations for a winter vacation in Europe**, tailored to different interests and budgets:

---

## **1. For Snow and Winter Sports Lovers**

### **1.1. The Swiss Alps (e.g., Zermatt, St. Moritz)**
- **Why:** World-class skiing, snowboarding, scenic villages, and luxury spas.
- **Highlights:** Matterhorn views, après-ski, Glacier Express train.

### **1.2. Chamonix, France**
- **Why:** Legendary skiing and snowboarding under Mont Blanc, vibrant town.
- **Highlights:** Winter hiking, cable cars, French alpine cuisine.

### **1.3. Austria (Innsbruck & Tyrol)**
- **Why:** Reliable snow, charming towns, traditi

## Step 6: Add Tools to Azure AI Foundry Agent

Azure AI Foundry agents support the same Python function tools as Azure OpenAI agents, plus additional built-in tools (code interpreter, file search) we'll explore in later tutorials.

In [6]:
# Define custom tools (same as Tutorial 01)
def get_weather(
    location: Annotated[str, Field(description="City or location name")],
) -> str:
    """Get current weather for a location."""
    # Simulated weather data
    conditions = ["sunny", "partly cloudy", "cloudy", "rainy", "stormy"]
    temp = randint(10, 35)
    condition = choice(conditions)
    return f"The weather in {location} is {condition} with a temperature of {temp}°C."

def convert_currency(
    amount: Annotated[float, Field(description="Amount to convert")],
    from_currency: Annotated[str, Field(description="Source currency (USD, EUR, GBP, etc.)")],
    to_currency: Annotated[str, Field(description="Target currency")],
) -> str:
    """Convert currency amounts using current exchange rates."""
    # Simulated exchange rates (as of October 2025)
    rates = {"USD": 1.0, "EUR": 0.92, "GBP": 0.79, "JPY": 149.50, "CAD": 1.36}
    from_currency, to_currency = from_currency.upper(), to_currency.upper()
    
    if from_currency not in rates or to_currency not in rates:
        return f"Sorry, unsupported currency: {from_currency} or {to_currency}"
    
    # Convert to USD first, then to target currency
    usd_amount = amount / rates[from_currency]
    converted = usd_amount * rates[to_currency]
    
    return f"{amount} {from_currency} = {converted:.2f} {to_currency}"

print(" Tools defined: get_weather, convert_currency")

 Tools defined: get_weather, convert_currency


In [7]:
# Create agent with tools
async def create_agent_with_tools():
    print(" Creating Azure AI Foundry agent with tools...\n")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            async_credential=credential,
            project_endpoint=project_endpoint,
            model_deployment_name=model_deployment
        ).create_agent(
            name="AzureTravelAssistantWithTools",
            instructions="""
            You are a travel assistant with access to real-time tools.
            Use the weather tool to check current conditions at destinations.
            Use the currency converter to help users with budgeting.
            Always provide helpful, specific advice.
            """,
            tools=[get_weather, convert_currency]  # Add Python function tools
        ) as agent,
    ):
        print(" Agent created with tools: get_weather, convert_currency\n")
        
        query = "What's the weather in Barcelona? Also, how much is 500 USD in Euros?"
        print(f" User: {query}\n")
        
        response = await agent.run(query)
        print(f" Agent: {response}")

# Run the agent
if project_endpoint and model_deployment:
    await create_agent_with_tools()
else:
    print("⚠️  Please configure your Azure AI Foundry project first (see Step 3)")

 Creating Azure AI Foundry agent with tools...

 Agent created with tools: get_weather, convert_currency

 User: What's the weather in Barcelona? Also, how much is 500 USD in Euros?

 Agent: The current weather in Barcelona is sunny with a temperature of 23°C.

As for the currency conversion, 500 USD is approximately 460 Euros. If you need more travel tips or details, let me know!


## Step 7: Streaming Responses

Azure AI Foundry agents support streaming for better user experience - responses appear progressively rather than all at once.

In [8]:
async def streaming_example():
    print(" Creating streaming agent...\n")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            async_credential=credential,
            project_endpoint=project_endpoint,
            model_deployment_name=model_deployment
        ).create_agent(
            name="StreamingTravelAgent",
            instructions="You are a helpful travel assistant. Provide detailed, enthusiastic responses.",
            tools=[get_weather]
        ) as agent,
    ):
        query = "Plan a 3-day itinerary for Paris, including weather info."
        print(f" User: {query}\n")
        print(" Agent: ", end="", flush=True)
        
        # Stream the response word by word
        async for chunk in agent.run_stream(query):
            if chunk.text:
                print(chunk.text, end="", flush=True)
        
        print("\n")  # New line at the end

if project_endpoint and model_deployment:
    await streaming_example()
else:
    print("⚠️  Please configure your Azure AI Foundry project first (see Step 3)")

 Creating streaming agent...

 User: Plan a 3-day itinerary for Paris, including weather info.

 Agent: Here's a bright and delightful 3-day Paris itinerary, perfect for sunny weather (around 16°C). Enjoy the charm of the City of Light!

Day 1: Classic Paris
- Morning: Start at the Eiffel Tower. Arrive early to beat the crowds and capture amazing photos in the sun!
- Midday: Walk along the Seine towards Musée d'Orsay—an art lover's paradise.
- Lunch: Enjoy a riverside bistro (try Café Campana inside the museum or Le Voltaire).
- Afternoon: Visit the Louvre Museum. Explore masterpieces like the Mona Lisa and stroll through the Tuileries Gardens.
- Evening: Wander through the historic Marais district—great for boutiques, vibrant cafés, and people-watching.

Day 2: Royal & Romantic Paris
- Morning: Head to Montmartre for views from Sacré-Cœur Basilica. The sunny weather makes the artistic streets extra picturesque.
- Midday: Have lunch at a classic Montmartre café, like Café des Deux Moul

## Step 8: Explicit Configuration (Without Environment Variables)

You can configure everything explicitly without relying on environment variables. This is useful when working with multiple Azure AI projects.

In [ ]:
async def explicit_config_example():
    """
    Example showing explicit configuration.
    Replace the values below with your actual Azure AI Foundry project details.
    """
    
    # Your Azure AI Foundry project details
    my_project_endpoint = "https://your-project.services.ai.azure.com/api/projects/your-project-name"
    my_model_deployment = "gpt-4"  # or gpt-4o, gpt-35-turbo
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            async_credential=credential,
            project_endpoint=my_project_endpoint,  # Explicit endpoint
            model_deployment_name=my_model_deployment,  # Explicit model
        ).create_agent(
            name="ExplicitConfigAgent",
            instructions="You are a helpful assistant.",
        ) as agent,
    ):
        response = await agent.run("What's the capital of France?")
        print(f"🤖 Agent: {response}")

# Uncomment and update the values in the function to run
# await explicit_config_example()

print(" To use explicit configuration:")
print("   1. Edit the function above with your actual values")
print("   2. Uncomment the last line")
print("   3. Run this cell")

## Step 9: Understanding Agent Lifecycle Management

### Ephemeral vs Persistent Agents

**Option 1: Ephemeral Agents (What we've been using)**

Using `async with`, agents are automatically created and deleted:

```python
async with client.create_agent(...) as agent:
    response = await agent.run("query")
# Agent is deleted here
```

✅ **Best for:** One-off queries, testing, temporary workflows  
❌ **Not ideal for:** Multi-turn conversations, reusing agents

**Option 2: Persistent Agents (For Production)**

Create an agent once and reuse it across multiple sessions:

```python
# Create once
async with AzureAIAgentClient(credential=credential) as client:
    agent = await client.create_agent(name="MyAgent", instructions="...")
    agent_id = agent.id  # Save this!
    
# Reuse many times (even in different sessions)
async with AzureAIAgentClient(credential=credential) as client:
    agent = await client.get_agent(agent_id)
    response = await agent.run("query")
```

✅ **Best for:** Production applications, multi-turn conversations  
✅ **Cost effective:** No repeated agent creation  
✅ **Maintains state:** Conversations persist

### When to Use Each:

- **Tutorials/Learning** → Ephemeral (automatic cleanup)
- **Testing** → Ephemeral
- **Production Apps** → Persistent
- **Multi-user Apps** → One persistent agent per user (or shared)

## Step 10: Side-by-Side Comparison

Let's compare **Azure OpenAI Direct** (Tutorial 01) vs **Azure AI Foundry** (this tutorial) using the same query.

In [ ]:
# Define a common test query
test_query = "What are the top 3 beach destinations in Europe for families?"

# Method 1: Azure OpenAI Direct (from Tutorial 01)
async def test_azure_openai_direct():
    from agent_framework.azure import AzureOpenAIChatClient
    
    print("=" * 60)
    print("METHOD 1: Azure OpenAI Direct (Tutorial 01)")
    print("=" * 60)
    
    # Check if we have Azure OpenAI credentials
    if not all([os.getenv("AZURE_OPENAI_ENDPOINT"), os.getenv("AZURE_OPENAI_API_KEY")]):
        print("⚠️  Azure OpenAI not configured (missing credentials from Tutorial 01)")
        return
    
    chat_client = AzureOpenAIChatClient(
        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        deployment_name=os.getenv("MODEL_DEPLOYMENT_NAME", "gpt-4")
    )
    
    agent = ChatAgent(
        chat_client=chat_client,
        name="DirectAzureOpenAI",
        instructions="You are a travel assistant."
    )
    
    print(f"👤 User: {test_query}\n")
    response = await agent.run(test_query)
    print(f"🤖 Agent: {str(response)[:200]}...\n")
    print(f"✅ Authentication: API Key")
    print(f"✅ Approach: Direct Azure OpenAI endpoint")
    print()

# Method 2: Azure AI Foundry (this tutorial)
async def test_azure_ai_foundry():
    print("=" * 60)
    print("METHOD 2: Azure AI Foundry (Tutorial 01b)")
    print("=" * 60)
    
    if not (project_endpoint and model_deployment):
        print("⚠️  Azure AI Foundry not configured")
        return
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            async_credential=credential,
            project_endpoint=project_endpoint,
            model_deployment_name=model_deployment
        ).create_agent(
            name="FoundryAgent",
            instructions="You are a travel assistant."
        ) as agent,
    ):
        print(f"👤 User: {test_query}\n")
        response = await agent.run(test_query)
        print(f"🤖 Agent: {str(response)[:200]}...\n")
        print(f"✅ Authentication: Azure AD (Entra ID)")
        print(f"✅ Approach: Azure AI Foundry platform")
        print()

# Run both comparisons
await test_azure_openai_direct()
await test_azure_ai_foundry()

print("=" * 60)
print("COMPARISON SUMMARY")
print("=" * 60)
print("Both methods use Azure OpenAI models, but with different approaches:")
print("• Tutorial 01: Simple, API key-based, direct access")
print("• Tutorial 01b: Enterprise-ready, Azure AD auth, full AI platform")

METHOD 1: Azure OpenAI Direct (Tutorial 01)
👤 User: What are the top 3 beach destinations in Europe for families?



TypeError: 'AgentRunResponse' object is not subscriptable

## 📊 Key Takeaways

### When to Use Azure AI Foundry:

✅ **Enterprise production workloads** - Need security, compliance, governance  
✅ **Team collaboration** - Multiple developers sharing projects and resources  
✅ **Advanced AI features** - Code interpreter, file search, Bing grounding  
✅ **Content safety** - Automatic harmful content filtering  
✅ **Full monitoring** - Azure Monitor integration with distributed tracing  
✅ **Managed identities** - No secrets management needed  
✅ **RBAC & VNET** - Fine-grained access control and network isolation

### When to Use Azure OpenAI Direct (Tutorial 01):

✅ **Quick prototyping** - Faster setup with just endpoint + API key  
✅ **Simple applications** - Don't need enterprise platform features  
✅ **Learning** - Easier to understand fundamentals  
✅ **Cost sensitivity** - No Azure AI Foundry project overhead  
✅ **Maximum control** - Direct API access for custom scenarios

### Technical Comparison:

| Aspect | Azure OpenAI Direct | Azure AI Foundry |
|--------|---------------------|------------------|
| **Import** | `AzureOpenAIChatClient` | `AzureAIAgentClient` |
| **Authentication** | API key (string) | Azure credential (object) |
| **Setup Complexity** | Low (2 values) | Medium (Azure project) |
| **Context Manager** | Optional | Recommended (`async with`) |
| **Agent Lifecycle** | Manual | Automatic or persistent |
| **Built-in Tools** | None (functions only) | Code interpreter, file search, Bing |
| **Monitoring** | External needed | Built-in Azure Monitor |
| **Cost** | Pay-per-token only | Pay-per-token + platform |

### Development Best Practice:

1. **Learn & Prototype** → Tutorial 01 (Azure OpenAI Direct)
2. **Enterprise Deploy** → Tutorial 01b (Azure AI Foundry)
3. **Choose based on needs** → Not one-size-fits-all!

Both approaches use the same underlying Azure OpenAI models - the difference is in the platform features and security model.

## 🔬 Practice Exercises

Try these exercises to deepen your understanding:

1. **Create your own agent** with custom instructions for a different domain (e.g., fitness coach, cooking assistant)

2. **Add a custom tool** - Create a function that provides real data (e.g., check stock prices, get news)

3. **Experiment with authentication** - Try using `DefaultAzureCredential` instead of `AzureCliCredential`

4. **Explore the portal** - Go to [ai.azure.com](https://ai.azure.com) and view your project, deployments, and usage

5. **Compare response times** - Time responses from Azure OpenAI Direct vs Azure AI Foundry

6. **Test streaming** - Modify the streaming example to use your own tools

In [ ]:
# 🎯 Your Practice Code Here!
# Try creating an agent with your own instructions and tools

# Example starter code:
async def my_custom_agent():
    """
    TODO: Customize this agent for your use case
    - Change the name and instructions
    - Add your own custom tools
    - Try different queries
    """
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            async_credential=credential,
            project_endpoint=project_endpoint,
            model_deployment_name=model_deployment
        ).create_agent(
            name="MyCustomAgent",
            instructions="Your custom instructions here...",
            # tools=[your_custom_tool]  # Add your tools
        ) as agent,
    ):
        response = await agent.run("Your custom query here...")
        print(f"🤖 {response}")

# Uncomment to run:
# if project_endpoint and model_deployment:
#     await my_custom_agent()

## 🎯 What's Next?

### Continue Your Learning Journey:

**Next Tutorial** → `02_agent_with_tools.ipynb`
- Deep dive into custom Python function tools
- Learn when and how agents call tools
- Handle tool errors and edge cases
- Build more complex tool interactions

### Explore Azure AI Foundry:

1. **Visit your project**: [ai.azure.com](https://ai.azure.com)
   - View deployments and usage metrics
   - Explore the playground
   - Check cost and consumption

2. **Try advanced features**:
   - Code Interpreter for data analysis
   - File Search for document Q&A
   - Bing Grounding for real-time info

3. **Read the docs**:
   - [Azure AI Foundry Documentation](https://learn.microsoft.com/azure/ai-studio/)
   - [Agent Framework Guide](https://learn.microsoft.com/agent-framework/)
   - [Azure OpenAI Service](https://learn.microsoft.com/azure/ai-services/openai/)

---

## 📋 Quick Reference

### Environment Variables (.env file)

```bash
# Azure AI Foundry (Tutorial 01b)
AZURE_AI_PROJECT_ENDPOINT=https://your-project.services.ai.azure.com/api/projects/your-project
AZURE_AI_MODEL_DEPLOYMENT_NAME=gpt-4

# Azure OpenAI Direct (Tutorial 01) 
AZURE_OPENAI_ENDPOINT=https://your-resource.openai.azure.com/
AZURE_OPENAI_API_KEY=your-api-key-here
MODEL_DEPLOYMENT_NAME=gpt-4
```

### Basic Azure AI Foundry Agent Pattern

```python
# Import
from agent_framework_azure_ai import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

# Create and use agent
async with (
    AzureCliCredential() as credential,
    AzureAIAgentClient(
        async_credential=credential,
        project_endpoint="your-endpoint",
        model_deployment_name="gpt-4"
    ).create_agent(
        name="MyAgent",
        instructions="Your instructions",
        tools=[your_tool]  # Optional
    ) as agent,
):
    response = await agent.run("Your query")
```

### Authentication Command

```bash
# Login to Azure (run this first!)
az login
```

---

**Ready to continue?** → Open `02_agent_with_tools.ipynb` 